### Import Libraries

In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import os
import shutil

### Load and Normalize the Dataset

In [31]:
# Load the dataset
file_path = 'D:\\Github Mikezxc\\Big-data-stock-real-time-platform\\merged_data_with_ma.csv'
merged_df = pd.read_csv(file_path)

# Normalize data
price_scaler = MinMaxScaler()
merged_df[['close']] = price_scaler.fit_transform(merged_df[['close']])

feature_scaler = MinMaxScaler()
merged_df[['MA30', 'MA90']] = feature_scaler.fit_transform(merged_df[['MA30', 'MA90']])


### Prepare Sequences for LSTM

In [32]:
# Prepare sequences for LSTM
def create_sequences(df, time_steps=30):
    sequences = []
    labels = []
    for i in range(len(df) - time_steps):
        sequence = df[['close', 'MA30', 'MA90']].iloc[i:i+time_steps].values
        label = df['close'].iloc[i+time_steps]
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)


### Define the LSTM HyperModel

In [33]:
# Define the LSTM HyperModel
class LSTMHyperModel(kt.HyperModel):
    def build(self, hp):
        model = Sequential()
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(LSTM(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), 
                           return_sequences=(i != hp.Int('num_layers', 1, 3) - 1), input_shape=(time_steps, 3)))
        model.add(Dense(1))  # Output layer should match the number of features
        model.compile(optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)),
            loss='mean_squared_error')
        return model


### Train and Evaluate the Model for Each Ticker

In [34]:
# Train and evaluate the model for each ticker
tickers = merged_df['ticker'].unique()

for ticker in tickers:
    ticker_df = merged_df[merged_df['ticker'] == ticker].dropna()
    
    # Create sequences and labels
    time_steps = 30
    X, y = create_sequences(ticker_df, time_steps)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Initialize RandomSearch Tuner
    tuner = kt.RandomSearch(
        LSTMHyperModel(),
        objective='val_loss',
        max_trials=20,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'lstm_stock_model_{ticker}'
    )

    # Perform hyperparameter search
    tuner.search(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

    # Retrieve the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"Ticker: {ticker}")
    print(f"The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')}.")
    for i in range(best_hps.get('num_layers')):
        print(f"Layer {i + 1}: {best_hps.get(f'units_{i}')} units")
    print(f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.")

    # Build the model with the optimal hyperparameters
    model = tuner.hypermodel.build(best_hps)

    # Train the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

    # Create directory for ticker if it doesn't exist
    os.makedirs(f'models/{ticker}', exist_ok=True)

    # Save the model
    model.save(f'models/{ticker}/lstm_stock_model_best_{ticker}.h5')

    # Save the scalers
    joblib.dump(price_scaler, f'models/{ticker}/{ticker}_price_scaler.pkl')
    joblib.dump(feature_scaler, f'models/{ticker}/{ticker}_feature_scaler.pkl')

    # Save the history for plotting
    with open(f'models/{ticker}/history_{ticker}.pkl', 'wb') as file:
        joblib.dump(history.history, file)

    # Make predictions
    y_pred = model.predict(X_test)

    # Inverse transform the predictions and the actual values
    y_pred = price_scaler.inverse_transform(y_pred)
    y_test = price_scaler.inverse_transform(y_test.reshape(-1, 1))

    # Save the predictions for plotting
    np.save(f'models/{ticker}/y_test_{ticker}.npy', y_test)
    np.save(f'models/{ticker}/y_pred_{ticker}.npy', y_pred)


Reloading Tuner from my_dir\lstm_stock_model_AAPL\tuner0.json
Ticker: AAPL
The hyperparameter search is complete. The optimal number of layers is 1.
Layer 1: 288 units
The optimal learning rate for the optimizer is 0.0020813994011954795.
Epoch 1/50


c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 0.0056 - val_loss: 3.1383e-04
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 1.0614e-04 - val_loss: 2.8333e-04
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 9.6405e-05 - val_loss: 1.8789e-04
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 6.4631e-05 - val_loss: 3.1058e-04
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 8.5358e-05 - val_loss: 1.4154e-04
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 6.3799e-05 - val_loss: 1.3408e-04
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 5.6681e-05 - val_loss: 2.1203e-04
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 4.6233e-05 - val_loss: 1.3712e-04
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 4.2935e-05 - val_loss: 1.3112e-04
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 3.7035e-05 - val_loss: 9.9787e-05
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 3.8700e-05 - val_loss: 1

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Reloading Tuner from my_dir\lstm_stock_model_GOOGL\tuner0.json
Ticker: GOOGL
The hyperparameter search is complete. The optimal number of layers is 3.
Layer 1: 32 units
Layer 2: 32 units
Layer 3: 64 units
The optimal learning rate for the optimizer is 0.0035065850163276276.
Epoch 1/50


c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 71ms/step - loss: 0.0328 - val_loss: 4.4881e-04
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - loss: 2.2955e-04 - val_loss: 2.8370e-04
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 1.6748e-04 - val_loss: 3.2619e-04
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 1.6840e-04 - val_loss: 4.6208e-04
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - loss: 2.0483e-04 - val_loss: 1.7385e-04
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 1.4974e-04 - val_loss: 1.5102e-04
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 1.2057e-04 - val_loss: 3.0696e-04
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 1.5532e-04 - val_loss: 1.8666e-04
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 9.9578e-05 - val_loss: 1.2808e-04
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 1.2192e-04 - val_loss: 1.2399e-04
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 1.0628e-04 - val_loss: 1

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
Reloading Tuner from my_dir\lstm_stock_model_TSLA\tuner0.json
Ticker: TSLA
The hyperparameter search is complete. The optimal number of layers is 3.
Layer 1: 160 units
Layer 2: 352 units
Layer 3: 160 units
The optimal learning rate for the optimizer is 0.00034786388011680107.
Epoch 1/50


c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - loss: 0.0136 - val_loss: 0.0063
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - loss: 0.0013 - val_loss: 0.0078
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 101ms/step - loss: 0.0013 - val_loss: 0.0018
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 90ms/step - loss: 9.9624e-04 - val_loss: 0.0051
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - loss: 9.2070e-04 - val_loss: 0.0014
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 91ms/step - loss: 7.4875e-04 - val_loss: 0.0019
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - loss: 8.5798e-04 - val_loss: 0.0011
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - loss: 5.9793e-04 - val_loss: 0.0021
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 91ms/step - loss: 7.4456e-04 - val_loss: 0.0018
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 90ms/step - loss: 5.1726e-04 - val_loss: 9.3106e-04
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 92ms/step - loss: 5.9489e-04 - val_loss: 0.0010
Epoch 12/50
60/60 ━━━━━━━━━━━━━━━━

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step


### Plotting the Results

In [35]:
# Plotting the training and validation loss
def plot_training_validation_loss(ticker):
    with open(f'models/{ticker}/history_{ticker}.pkl', 'rb') as file:
        history = joblib.load(file)

    plt.figure(figsize=(14, 5))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {ticker}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'models/{ticker}/training_validation_loss_{ticker}.png')  # Save the plot
    plt.close()

# Plotting the stock price prediction
def plot_stock_price_prediction(ticker):
    y_test = np.load(f'models/{ticker}/y_test_{ticker}.npy')
    y_pred = np.load(f'models/{ticker}/y_pred_{ticker}.npy')

    plt.figure(figsize=(14, 5))
    plt.plot(y_test, color='blue', label='Actual Stock Price')
    plt.plot(y_pred, color='red', label='Predicted Stock Price')
    plt.title(f'Stock Price Prediction for {ticker}')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.savefig(f'models/{ticker}/stock_price_prediction_{ticker}.png')  # Save the plot
    plt.close()

# Plot results for each ticker
for ticker in tickers:
    plot_training_validation_loss(ticker)
    plot_stock_price_prediction(ticker)


In [37]:
# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Ensure the directory exists
os.makedirs(f'my_dir/lstm_stock_model_{ticker}', exist_ok=True)

# Print and save hyperparameters
with open(f'my_dir/lstm_stock_model_{ticker}/best_hyperparameters_{ticker}.txt', 'w') as f:
    f.write(f"Stock: {ticker}\n")
    f.write(f"The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')}.\n")
    for i in range(best_hps.get('num_layers')):
        f.write(f"Layer {i + 1}: {best_hps.get(f'units_{i}')} units\n")
    f.write(f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.\n")

print(f"Stock: {ticker}")
print(f"The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')}.")
for i in range(best_hps.get('num_layers')):
    print(f"Layer {i + 1}: {best_hps.get(f'units_{i}')} units")
print(f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.")

Stock: TSLA
The hyperparameter search is complete. The optimal number of layers is 3.
Layer 1: 160 units
Layer 2: 352 units
Layer 3: 160 units
The optimal learning rate for the optimizer is 0.00034786388011680107.


In [38]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import joblib

# Function to calculate and save metrics
def calculate_metrics(y_true, y_pred, ticker):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    
    accuracy = np.mean(np.abs((y_true - y_pred) / y_true) < 0.05) * 100  # Within 5% tolerance
    
    metrics = (
        f"Metrics for {ticker}:\n"
        f"Mean Squared Error (MSE): {mse}\n"
        f"Root Mean Squared Error (RMSE): {rmse}\n"
        f"R-squared (R2): {r2}\n"
        f"Mean Absolute Error (MAE): {mae}\n"
        f"Accuracy (within 5% tolerance): {accuracy}%\n"
        "--------------------------\n"
    )
    
    # Print metrics to console
    print(metrics)
    
    # Save metrics to a file
    with open(f'models/{ticker}/metrics_{ticker}.txt', 'w') as file:
        file.write(metrics)

# Plotting the training and validation loss
def plot_training_validation_loss(ticker):
    with open(f'models/{ticker}/history_{ticker}.pkl', 'rb') as file:
        history = joblib.load(file)

    plt.figure(figsize=(14, 5))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {ticker}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'models/{ticker}/training_validation_loss_{ticker}.png')  # Save the plot
    plt.close()

# Plotting the stock price prediction
def plot_stock_price_prediction(ticker):
    y_test = np.load(f'models/{ticker}/y_test_{ticker}.npy')
    y_pred = np.load(f'models/{ticker}/y_pred_{ticker}.npy')

    # Calculate and save metrics
    calculate_metrics(y_test, y_pred, ticker)

    plt.figure(figsize=(14, 5))
    plt.plot(y_test, color='blue', label='Actual Stock Price')
    plt.plot(y_pred, color='red', label='Predicted Stock Price')
    plt.title(f'Stock Price Prediction for {ticker}')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.savefig(f'models/{ticker}/stock_price_prediction_{ticker}.png')  # Save the plot
    plt.close()


# Plot results for each ticker
for ticker in tickers:
    plot_training_validation_loss(ticker)
    plot_stock_price_prediction(ticker)


Metrics for AAPL:
Mean Squared Error (MSE): 15.886690410132681
Root Mean Squared Error (RMSE): 3.9858111357831145
R-squared (R2): 0.9640448481621426
Mean Absolute Error (MAE): 3.188053172007328
Accuracy (within 5% tolerance): 97.05882352941177%
--------------------------

Metrics for GOOGL:
Mean Squared Error (MSE): 39.302001219597514
Root Mean Squared Error (RMSE): 6.269130818510451
R-squared (R2): 0.9442706821393136
Mean Absolute Error (MAE): 5.342826390421891
Accuracy (within 5% tolerance): 67.85714285714286%
--------------------------

Metrics for MSFT:
Mean Squared Error (MSE): 136.99382544002432
Root Mean Squared Error (RMSE): 11.704436143617698
R-squared (R2): 0.9723947661063579
Mean Absolute Error (MAE): 9.681149876698727
Accuracy (within 5% tolerance): 89.70588235294117%
--------------------------

Metrics for AMZN:
Mean Squared Error (MSE): 30.832509203819694
Root Mean Squared Error (RMSE): 5.552702873720121
R-squared (R2): 0.9705591064443696
Mean Absolute Error (MAE): 4.3460